<a href="https://colab.research.google.com/github/thaisbeham/Applied_deep_learning/blob/main/Assigment2_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

import pandas as pd
#from utils import merge_datasets, smaller_set
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt

import os

In [3]:
fake_news = pd.read_csv("/content/drive/MyDrive/dados_/fake.csv")
fake_news.text=fake_news.text.astype(str)
real_news = pd.read_csv("/content/drive/MyDrive/dados_/df_2016.csv")
real_news.sentence =real_news.sentence.astype(str)

In [4]:
def merge_datasets(real, fake):
    #take unique values from real data
    real = real.drop_duplicates(subset = ['sentence'])

    # take 13000 rows from real data
    real = real.sample(n = 13000, random_state= 2, axis = 0)

    # add column label
    real['label'] = 1
    fake['label'] = 0

    real = real[['sentence', 'label']]
    fake = fake[['text', 'label']]
    real.rename(columns = {'sentence': 'text'}, inplace = True)


    #merge real and fake
    # use only the sentence column on fake
    merged = pd.DataFrame(real.append(fake, ignore_index = True))
    merged = merged.sample(frac = 1, random_state= 1, ignore_index = True)#.reset_index()

    return merged


In [5]:
df = merge_datasets(real_news, fake_news)

# train test split
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.15, random_state = 1)

In [6]:
train.head()

,text,label
14442,"New books by David Arnold, Laure Eve, Lauren O...",1
2505,Requiring women to register for the draft woul...,1
19267,‹ › Arnaldo Rodgers is a trained and educated ...,0
4172,Abby Martin Exposes What Hillary Clinton Reall...,0
8220,"November 4, 2016 - Fort Russ News - RusVesna ...",0


In [7]:
X = train['text']#.astype(string)
Y = train['label'].astype(int)

#X = pd.DataFrame(X, dtype='string')


In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15, random_state=42)

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [13]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [14]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [15]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [16]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
118/118 [==============================] - 9s 20ms/step - loss: 0.2017 - accuracy: 0.9267 - val_loss: 0.1086 - val_accuracy: 0.9657
Epoch 2/10
118/118 [==============================] - 2s 14ms/step - loss: 0.1020 - accuracy: 0.9638 - val_loss: 0.0951 - val_accuracy: 0.9665
Epoch 3/10
118/118 [==============================] - 2s 14ms/step - loss: 0.0818 - accuracy: 0.9708 - val_loss: 0.0907 - val_accuracy: 0.9675
Epoch 4/10
118/118 [==============================] - 2s 14ms/step - loss: 0.0717 - accuracy: 0.9750 - val_loss: 0.0901 - val_accuracy: 0.9699
Epoch 5/10
118/118 [==============================] - 2s 14ms/step - loss: 0.0645 - accuracy: 0.9766 - val_loss: 0.0870 - val_accuracy: 0.9713
Epoch 6/10
118/118 [==============================] - 2s 14ms/step - loss: 0.0586 - accuracy: 0.9787 - val_loss: 0.0858 - val_accuracy: 0.9689
Epoch 7/10
118/118 [==============================] - 2s 14ms/step - loss: 0.0535 - accuracy: 0.9805 - val_loss: 0.0896 - val_accuracy: 0.9707

In [17]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [18]:
accr = model.evaluate(test_sequences_matrix,Y_test)

104/104 [==============================] - 1s 5ms/step - loss: 0.1364 - accuracy: 0.9551


In [19]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.136
  Accuracy: 0.955


In [20]:
predict_labels = model.predict(test_sequences_matrix) 

104/104 [==============================] - 1s 4ms/step


In [28]:
# convert the range predicition into 0s or 1s
preds3=[ 1 if i>0.5 else 0 for i in predict_labels  ] 

print("sum",sum(preds3))
print("length",len(preds3))

sum 1658
length 3315


In [30]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, preds3, target_names = ['class 0', 'class 1']))


              precision    recall  f1-score   support

     class 0       0.96      0.95      0.96      1670
     class 1       0.95      0.96      0.95      1645

    accuracy                           0.96      3315
   macro avg       0.96      0.96      0.96      3315
weighted avg       0.96      0.96      0.96      3315



In [31]:
#Calculate confusion matrix
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(Y_test,preds3,labels=[0,1])
cm=pd.DataFrame(matrix,index=['class_0 pred','class_1 pred'],columns=['class_0 True','class_1 True'])
print(cm)

              class_0 True  class_1 True
class_0 pred          1589            81
class_1 pred            68          1577
